## Application of BPR on Movielens

In [ ]:
#%load_ext watermark
%load_ext autoreload 
%autoreload 2

In [ ]:
import os
import sys
import itertools
import numpy as np
import pandas as pd
from subprocess import call
from pybpr import *
import matplotlib.pyplot as plt
from functools import partial

In [ ]:
%%time
#df = load_movielens_data('ml-1m')
df = load_movielens_data('ml-100k')
dfshort = df[df['rating'] > 0]
df.head()

In [ ]:
cf = UserItemInteractions(
    name='MovieLens-100k',
    users=dfshort['user_id'],
    items=dfshort['item_id'],
    min_num_rating_per_user=10,
    min_num_rating_per_item=10,
    num_cores = 1
)
cf.print_memory_usage()
cf.generate_train_test(user_test_ratio=0.2)

In [ ]:
bpr = BPR(
    num_features=10,
    reg_lambda=0.,
    num_iters=50,
    learning_rate = 0.5,
    batch_size=5,
    initial_std=0.01,
    seed=None
)
bpr_ndcg_func = partial(
    cf.get_ndcg_metric,
    num_items=10
)
bpr.fit(cf.R_train, bpr_ndcg_func)
ndcg_df = pd.DataFrame(bpr.ndcg_metric)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,3))
ax[0].plot(ndcg_df['test'])
ax[1].plot(ndcg_df['train'])

In [ ]:
#bpr_ndcg_func(test=True), bpr_ndcg_func(test=False)

In [ ]:
ndcg_df